In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print(pd.__version__)

2.3.0


In [2]:
reviewsbmo = pd.read_json('reviews/com.bmo.mobile.json')
reviewsbmo.insert(0, 'bank', 'BMO')
reviewscibc = pd.read_json('reviews/com.cibc.android.mobi.json')
reviewscibc.insert(0,'bank', 'CIBC')
reviewsrbc = pd.read_json('reviews/com.rbc.mobile.android.json')
reviewsrbc.insert(0, 'bank', 'RBC')
reviewssb = pd.read_json('reviews/com.scotiabank.banking.json')
reviewssb.insert(0, 'bank', 'SCOTIA BANK')
reviewstd = pd.read_json('reviews/com.td.json')
reviewstd.insert(0, 'bank', 'TD')

In [3]:
all_reviews = pd.concat([reviewsbmo,reviewscibc,reviewsrbc,reviewssb,reviewstd], ignore_index = True)

In [4]:
all_reviews

,bank,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,BMO,c1e46217-4ef1-410b-811f-4376800cf5a9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,not any use,1,0,6.20.2,2025-06-30 22:09:09,"Hi Arran, thanks for your review. We're sorry ...",2025-07-02 11:48:06,6.20.2
1,BMO,0aa7be3b-9527-458e-96d5-3e69abbcf7cf,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I tried for 3days to access my account but bmo...,1,1,6.20.2,2025-06-30 21:01:44,"Hi Norbert, we're sorry to learn of your exper...",2025-07-02 09:18:46,6.20.2
2,BMO,97f17b6b-9759-4388-8bf5-71b7f7e2c42d,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,great,5,0,6.20.2,2025-06-30 16:21:41,"Hi there, thank you for your review and rating...",2025-07-02 08:56:59,6.20.2
3,BMO,832aea75-58b0-4c8f-b553-eccd647c9299,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,GARBAGE. AND EVERY PERSON THAT WORKS AT AND FO...,1,1,None,2025-06-30 15:23:11,Hi Jessica. We're sorry to learn our services ...,2025-07-02 08:55:23,None
4,BMO,e940c7c4-3fbc-467c-a379-87680df7c93b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,easy to install. the layout is awesome. no tro...,5,1,6.20.2,2025-06-28 17:25:17,Hi Cathy! Thanks for your review and 5-stars! ...,2025-07-02 08:54:23,6.20.2
...,...,...,...,...,...,...,...,...,...,...,...,...
21300,TD,b9e18543-63bc-4fe2-9b65-4a04b15a753b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,stopped working on my work phone. had to trans...,3,0,23.2.0,2023-05-31 10:16:22,None,None,23.2.0
21301,TD,fc559a89-7768-4924-ad8c-f645556821ca,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This app is horrible. Wasted 2 hours on the ph...,1,1,23.3.0,2023-05-31 08:02:10,None,None,23.3.0
21302,TD,3c6bffac-16a9-4353-ae8b-d1062c829906,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Always saying NO INTERNET wherever on mobile d...,1,0,23.2.0,2023-05-31 07:21:35,None,None,23.2.0
21303,TD,a1144dde-78dc-46ba-9226-3325f3a60759,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,T.D is just superior to all others,5,0,23.3.0,2023-05-31 06:44:32,None,None,23.3.0


In [5]:
reviewsclean = all_reviews.drop(columns = ['reviewId','userName', 'userImage', 'appVersion'])

In [6]:
reviewsclean = reviewsclean.dropna(subset = ['reviewCreatedVersion'])

In [7]:
reviewsclean

,bank,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,BMO,not any use,1,0,6.20.2,2025-06-30 22:09:09,"Hi Arran, thanks for your review. We're sorry ...",2025-07-02 11:48:06
1,BMO,I tried for 3days to access my account but bmo...,1,1,6.20.2,2025-06-30 21:01:44,"Hi Norbert, we're sorry to learn of your exper...",2025-07-02 09:18:46
2,BMO,great,5,0,6.20.2,2025-06-30 16:21:41,"Hi there, thank you for your review and rating...",2025-07-02 08:56:59
4,BMO,easy to install. the layout is awesome. no tro...,5,1,6.20.2,2025-06-28 17:25:17,Hi Cathy! Thanks for your review and 5-stars! ...,2025-07-02 08:54:23
5,BMO,convenient fast and reliable,5,1,6.20.2,2025-06-28 14:17:35,"Hi Shanee, thank you so much for your rating a...",2025-07-02 08:53:45
...,...,...,...,...,...,...,...,...
21300,TD,stopped working on my work phone. had to trans...,3,0,23.2.0,2023-05-31 10:16:22,None,None
21301,TD,This app is horrible. Wasted 2 hours on the ph...,1,1,23.3.0,2023-05-31 08:02:10,None,None
21302,TD,Always saying NO INTERNET wherever on mobile d...,1,0,23.2.0,2023-05-31 07:21:35,None,None
21303,TD,T.D is just superior to all others,5,0,23.3.0,2023-05-31 06:44:32,None,None


In [8]:
reviewsclean.to_csv('all_reviews.csv')